In [331]:
import pandas as pd
import networkx as nx
import requests

from sklearn.cluster import KMeans

In [332]:
resp = requests.get('https://data.montgomerycountymd.gov/resource/mmzv-x632.json?$limit=5100')

In [333]:
resp.status_code

200

In [334]:
resp.headers['content-type']

'application/json;charset=utf-8'

In [335]:
resp_obj = resp.json() #Convert output to JSON

resp_obj

[{'report_number': 'MCP2850002Q',
  'local_case_number': '230070299',
  'agency_name': 'Montgomery County Police',
  'acrs_report_type': 'Property Damage Crash',
  'crash_date_time': '2023-12-06T21:54:00.000',
  'route_type': 'County',
  'road_name': 'GOSHEN RD',
  'cross_street_type': 'Maryland (State)',
  'cross_street_name': 'MIDCOUNTY HWY',
  'municipality': 'N/A',
  'collision_type': 'HEAD ON',
  'weather': 'CLEAR',
  'surface_condition': 'DRY',
  'light': 'DARK LIGHTS ON',
  'traffic_control': 'TRAFFIC SIGNAL',
  'driver_substance_abuse': 'N/A',
  'person_id': '209D3ACD-1995-42DC-917A-4BC3B984417B',
  'driver_at_fault': 'No',
  'injury_severity': 'NO APPARENT INJURY',
  'circumstance': 'N/A',
  'driver_distracted_by': 'NOT DISTRACTED',
  'drivers_license_state': 'MD',
  'vehicle_id': 'FE937E17-A803-41A8-914A-0A24AA2008C5',
  'vehicle_damage_extent': 'DESTROYED',
  'vehicle_first_impact_location': 'TWELVE OCLOCK',
  'vehicle_second_impact_location': 'TWELVE OCLOCK',
  'vehicle_bod

In [336]:
df = pd.DataFrame(resp_obj) #Convert JSON response to DataFrame

df

,report_number,local_case_number,agency_name,acrs_report_type,crash_date_time,route_type,road_name,cross_street_type,cross_street_name,municipality,...,:@computed_region_kbsp_ykn9,latitude,:@computed_region_d7bw_bq6x,longitude,:@computed_region_rbt8_3x7n,geolocation,:@computed_region_6vgr_duib,off_road_description,related_non_motorist,non_motorist_substance_abuse
0,MCP2850002Q,230070299,Montgomery County Police,Property Damage Crash,2023-12-06T21:54:00.000,County,GOSHEN RD,Maryland (State),MIDCOUNTY HWY,N/A,...,36,39.15801333,35,-77.192055,1,"{'latitude': '39.15801333', 'longitude': '-77....",3,NaN,NaN,NaN
1,MCP2850002Q,230070299,Montgomery County Police,Property Damage Crash,2023-12-06T21:54:00.000,County,GOSHEN RD,Maryland (State),MIDCOUNTY HWY,N/A,...,36,39.15801333,35,-77.192055,1,"{'latitude': '39.15801333', 'longitude': '-77....",3,NaN,NaN,NaN
2,MCP32630044,230070281,Montgomery County Police,Property Damage Crash,2023-12-06T19:05:00.000,County,COMMERCE LA,Maryland (State),WISCONSIN AVE,N/A,...,12,38.98591127,111,-77.09493278,1,"{'latitude': '38.98591127', 'longitude': '-77....",6,NaN,NaN,NaN
3,MCP3197003V,230070260,Montgomery County Police,Property Damage Crash,2023-12-06T17:53:00.000,County,BEL PRE RD,Maryland (State),LAYHILL RD,N/A,...,59,39.09012167,6,-77.04489,1,"{'latitude': '39.09012167', 'longitude': '-77....",2,NaN,NaN,NaN
4,MCP3197003V,230070260,Montgomery County Police,Property Damage Crash,2023-12-06T17:53:00.000,County,BEL PRE RD,Maryland (State),LAYHILL RD,N/A,...,59,39.09012167,6,-77.04489,1,"{'latitude': '39.09012167', 'longitude': '-77....",2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,MCP3361000V,230053405,Montgomery County Police,Property Damage Crash,2023-09-06T10:15:00.000,Maryland (State),UNIVERSITY BLVD W,County,REEDIE DR,N/A,...,16,39.03962117,17,-77.0427825,1,"{'latitude': '39.03962117', 'longitude': '-77....",2,NaN,NaN,NaN
5096,MCP2837005K,230053401,Montgomery County Police,Property Damage Crash,2023-09-06T09:48:00.000,County,TRAVILAH RD,County,TRAVILAH TERR,N/A,...,22,39.06150833,53,-77.28067833,1,"{'latitude': '39.06150833', 'longitude': '-77....",6,NaN,NaN,NaN
5097,MCP2837005K,230053401,Montgomery County Police,Property Damage Crash,2023-09-06T09:48:00.000,County,TRAVILAH RD,County,TRAVILAH TERR,N/A,...,22,39.06150833,53,-77.28067833,1,"{'latitude': '39.06150833', 'longitude': '-77....",6,NaN,NaN,NaN
5098,MCP24870038,230053391,Montgomery County Police,Injury Crash,2023-09-06T08:34:00.000,County,BUDD RD,Maryland (State),WHITES FERRY RD,N/A,...,34,39.13417333,97,-77.40259833,16,"{'latitude': '39.13417333', 'longitude': '-77....",5,NaN,BICYCLIST,NONE DETECTED


In [337]:
#Sort to pull additional "@" columns to front of df
sort_df = df[sorted(df)]

#Slice off "@" columns
df = sort_df.iloc[:,8:]
df = df.fillna('')

df

,acrs_report_type,agency_name,circumstance,collision_type,crash_date_time,cross_street_name,cross_street_type,driver_at_fault,driver_distracted_by,driver_substance_abuse,...,vehicle_damage_extent,vehicle_first_impact_location,vehicle_going_dir,vehicle_id,vehicle_make,vehicle_model,vehicle_movement,vehicle_second_impact_location,vehicle_year,weather
0,Property Damage Crash,Montgomery County Police,N/A,HEAD ON,2023-12-06T21:54:00.000,MIDCOUNTY HWY,Maryland (State),No,NOT DISTRACTED,N/A,...,DESTROYED,TWELVE OCLOCK,North,FE937E17-A803-41A8-914A-0A24AA2008C5,HOND,SD,ACCELERATING,TWELVE OCLOCK,2015,CLEAR
1,Property Damage Crash,Montgomery County Police,N/A,HEAD ON,2023-12-06T21:54:00.000,MIDCOUNTY HWY,Maryland (State),Yes,TALKING OR LISTENING TO CELLULAR PHONE,N/A,...,DESTROYED,TWELVE OCLOCK,South,66C48EB3-4864-41B1-B4BE-C91FB10BC222,HOND,4S,ACCELERATING,TWELVE OCLOCK,2018,CLEAR
2,Property Damage Crash,Montgomery County Police,N/A,SAME DIRECTION RIGHT TURN,2023-12-06T19:05:00.000,WISCONSIN AVE,Maryland (State),Yes,LOOKED BUT DID NOT SEE,NONE DETECTED,...,SUPERFICIAL,THREE OCLOCK,South,37D28D1A-BEFF-43D8-9AA3-9075F3107EB4,NFLY,548,MAKING RIGHT TURN,THREE OCLOCK,2022,CLEAR
3,Property Damage Crash,Montgomery County Police,N/A,SAME DIRECTION SIDESWIPE,2023-12-06T17:53:00.000,LAYHILL RD,Maryland (State),No,NOT DISTRACTED,NONE DETECTED,...,FUNCTIONAL,ELEVEN OCLOCK,East,C5ADFE06-1E25-4135-8F3B-27715D2F0714,FORD,ESCAPE,MAKING RIGHT TURN,ELEVEN OCLOCK,2017,CLEAR
4,Property Damage Crash,Montgomery County Police,N/A,SAME DIRECTION SIDESWIPE,2023-12-06T17:53:00.000,LAYHILL RD,Maryland (State),Yes,LOOKED BUT DID NOT SEE,NONE DETECTED,...,FUNCTIONAL,TWO OCLOCK,West,718DBD97-F39D-4DBB-92D6-F2C3350394F0,BMW,330I,MAKING LEFT TURN,TWO OCLOCK,2018,CLEAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,Property Damage Crash,Montgomery County Police,N/A,SINGLE VEHICLE,2023-09-06T10:15:00.000,REEDIE DR,County,Yes,NOT DISTRACTED,NONE DETECTED,...,DISABLING,TWELVE OCLOCK,East,0BF02D76-0BE7-4064-8B27-6282E51D47F5,TOYOTA,SIENNA LE,MOVING CONSTANT SPEED,ELEVEN OCLOCK,2014,CLEAR
5096,Property Damage Crash,Montgomery County Police,N/A,OTHER,2023-09-06T09:48:00.000,TRAVILAH TERR,County,Yes,LOOKED BUT DID NOT SEE,NONE DETECTED,...,SUPERFICIAL,SIX OCLOCK,South,4FCCCF39-E608-4EBE-B85F-DFA9ACE4F7CF,MACK,GRANITE,BACKING,SIX OCLOCK,2023,N/A
5097,Property Damage Crash,Montgomery County Police,N/A,OTHER,2023-09-06T09:48:00.000,TRAVILAH TERR,County,No,NOT DISTRACTED,N/A,...,DISABLING,TWELVE OCLOCK,North,94E0D4A3-E636-434C-863B-E7310C9BFB38,AUDI,2S,STOPPED IN TRAFFIC LANE,TWELVE OCLOCK,2013,N/A
5098,Injury Crash,Montgomery County Police,N/A,STRAIGHT MOVEMENT ANGLE,2023-09-06T08:34:00.000,WHITES FERRY RD,Maryland (State),Yes,LOOKED BUT DID NOT SEE,NONE DETECTED,...,SUPERFICIAL,ELEVEN OCLOCK,North,40248645-E554-4A76-9A56-F4846E89A7CB,SUBA,CROSSTREK,STARTING FROM LANE,ELEVEN OCLOCK,2019,CLEAR


In [338]:
report_dict = {}

for index, row in df.iterrows():
    road_count = {}
    road_count[row['road_name']] = 1
    report_dict[row['report_number']] = road_count
    print(report_dict[row['report_number']])

{'GOSHEN RD': 1}
{'GOSHEN RD': 1}
{'COMMERCE LA': 1}
{'BEL PRE RD': 1}
{'BEL PRE RD': 1}
{'PARTNERSHIP RD': 1}
{'PARTNERSHIP RD': 1}
{'GEORGIA AVE': 1}
{'GEORGIA AVE': 1}
{'GEORGIA AVE': 1}
{'GEORGIA AVE': 1}
{'GEORGIA AVE': 1}
{'RANDOLPH RD': 1}
{'RANDOLPH RD': 1}
{'E MONTGOMERY AVE': 1}
{'E MONTGOMERY AVE': 1}
{'VEIRS MILL RD': 1}
{'VEIRS MILL RD': 1}
{'OLD COLUMBIA PIKE': 1}
{'OLD COLUMBIA PIKE': 1}
{'QUINCE ORCHARD RD': 1}
{'QUINCE ORCHARD RD': 1}
{'ROCKVILLE PIKE': 1}
{'ROCKVILLE PIKE': 1}
{'NORBECK RD': 1}
{'NORBECK RD': 1}
{'NORBECK RD': 1}
{'FREDERICK RD': 1}
{'FREDERICK RD': 1}
{'FALLS RD': 1}
{'FALLS RD': 1}
{'FALLS RD': 1}
{'FREDERICK RD': 1}
{'FREDERICK RD': 1}
{'ROCKVILLE PIKE': 1}
{'ROCKVILLE PIKE': 1}
{'GOSHEN RD': 1}
{'GOSHEN RD': 1}
{'': 1}
{'': 1}
{'WINGED ELM DR': 1}
{'WINGED ELM DR': 1}
{'': 1}
{'EAST WEST HWY': 1}
{'MIDDLEBROOK RD': 1}
{'MIDDLEBROOK RD': 1}
{'DUFIEF DR': 1}
{'DUFIEF DR': 1}
{'SNOWDEN FARM PKWY': 1}
{'SNOWDEN FARM PKWY': 1}
{'': 1}
{'NEW HAMPSHIRE A

In [339]:
index = report_dict.keys()

rows = [report_dict[k] for k in index]

df1 = pd.DataFrame(rows, index=index)

df1 = df1.fillna(0)

df1

,GOSHEN RD,COMMERCE LA,BEL PRE RD,PARTNERSHIP RD,GEORGIA AVE,RANDOLPH RD,E MONTGOMERY AVE,VEIRS MILL RD,OLD COLUMBIA PIKE,QUINCE ORCHARD RD,...,APPLE AVE,ELLINGTON BLVD,HOPKINS RD,OAKVALE ST,MILLER FALL RD,KERSEY LA,KINGWOOD DR,GROSVENOR LA,QUEENAIR DR,BUDD RD
MCP2850002Q,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MCP32630044,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MCP3197003V,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MCP2356003G,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MCP33340022,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MCP22980148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MCP3361000V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MCP2837005K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MCP24870038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [340]:
k=10

In [341]:
cluster_model = KMeans(n_clusters=k)
cluster_model.fit(df1)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=10)

In [342]:
cluster_labels = cluster_model.predict(df1)
report_cluster_df = pd.DataFrame(cluster_labels, index=df1.index, columns=["cluster"])
report_cluster_df["cluster"].value_counts()

cluster
5    1984
1     325
9     148
2     102
6      90
4      68
0      66
7      35
8      33
3      28
Name: count, dtype: int64

In [343]:
for cluster,reports in report_cluster_df.groupby("cluster"):
    print("Cluster:", cluster, "Size:", reports.shape[0])
    
    for r_id in reports.sample(5).index:
        print("\t", r_id, list(report_dict[r_id].keys())[0])

Cluster: 0 Size: 66
	 MCP3269002T COLUMBIA PIKE
	 MCP33290022 COLUMBIA PIKE
	 MCP3227004V COLUMBIA PIKE
	 MCP3207003M COLUMBIA PIKE
	 MCP3325002D COLUMBIA PIKE
Cluster: 1 Size: 325
	 EJ7873008F 
	 MCP27650069 
	 MCP30910043 
	 MCP28980044 
	 MCP3256002H 
Cluster: 2 Size: 102
	 EJ7781004K FREDERICK RD
	 EJ7833007V FREDERICK RD
	 DD5603009W FREDERICK RD
	 DD5644003D FREDERICK RD
	 DD5660000M FREDERICK RD
Cluster: 3 Size: 28
	 EJ7895004Q DARNESTOWN RD
	 MCP2583003Q DARNESTOWN RD
	 MCP26370057 DARNESTOWN RD
	 MCP2583003H DARNESTOWN RD
	 MCP21210030 DARNESTOWN RD
Cluster: 4 Size: 68
	 MCP2844006X CONNECTICUT AVE
	 MCP292300B3 CONNECTICUT AVE
	 MCP2723007R CONNECTICUT AVE
	 MCP3161005Q CONNECTICUT AVE
	 MCP304800BT CONNECTICUT AVE
Cluster: 5 Size: 1984
	 MCP2761004P KINGS CROSSING BLVD
	 MCP2016007D TWINBROOK PKWY
	 HA24850013 SLIGO CREEK PKWY
	 MCP3350001F HILLERY WAY
	 MCP2780007C GERMANTOWN RD
Cluster: 6 Size: 90
	 MCP3136006P NEW HAMPSHIRE AVE
	 DM8522000Z NEW HAMPSHIRE AVE
	 DM8352000N 